In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# .env 로드 (상위 디렉토리에서 자동 탐색)
load_dotenv()  # 또는 load_dotenv(dotenv_path='../.env')
api_key = os.getenv('OPENAI_API_KEY')
print('API Key loaded' if api_key else 'API Key not found!')

# ===== 프롬프트 구성 (안정성 중시형 · 보통퀄리티 4/5) =====
persona = """
당신은 금융권 대규모 시스템을 보수적으로 안정적으로 구축해온 시니어 엔터프라이즈 아키텍트입니다(경력 10년+). 
검증된 상용/오픈소스 기술의 안전한 조합, 위험 식별·완화, 품질보증 체계(테스트/검수/표준 산출물)와 체계적 사업 관리에 강점이 있습니다. 
과도한 혁신보다 일관성과 재현 가능한 품질을 중시합니다.
"""

concept = """
이번 제안은 '안정성 중시형' 컨셉입니다. 가장 널리 검증된 안정적인 기술 스택을 사용하고, 리스크 관리, 품질 보증, 체계적인 사업 관리 방안을 강조하여 고객에게 신뢰를 주는 데 집중하세요.
"""

# 퀄리티(4/5) 가정: RFP 요구사항을 모두 충족하지 못하고, 서술의 구체성/근거가 부족한 수준으로 유도
quality_constraints = """
[품질 제약(보통 4/5)]
- RFP 전 항목을 모두 제안하지 마세요(일부 항목은 모호하거나 생략된 형태로 남겨두기).
- 근거 수치/세부 설계/테스트 기준은 최소화하여 개괄 수준으로 작성.
- 사례·유즈케이스는 간단 요약 수준으로만 제시(상세 시나리오/데이터 흐름 생략).
- 일정·조직·KPI는 범위만 대략 제시(정확 수치/역할 세분화 미흡).
- 규정·보안·컴플라이언스는 ‘준수 예정’ 수준의 원론적 표현을 사용.
"""

# 신한투자증권 로그정보시스템 고도화 RFP 컨텍스트 요약 (필수 요구)
goal_context = """
당신은 지금 {신한투자증권 로그정보시스템 고도화} 사업의 입찰 경쟁에 참여하고 있습니다.
가. 사업명: 『신한투자증권 로그정보시스템 고도화』
나. 사업 주요 내용(안)
  (1) 사업 취지
    - 메타 차세대 프로젝트와 연계된 신규 로그를 사용자가 이해하기 쉬운 웹 UI로 조회 가능하도록 시스템 고도화
  (2) 추진 목적
    - 사용자 중심 UX로 로그 탐색·검색·필터링 경험 개선
    - 컴플라이언스 이슈 대응: 민감정보 항목 승인/권한 통제 및 감사 추적 강화
    - 그룹 권한(IT/일반직원 등)에 따른 로그 조회 제한과 차등 정보 노출
  (3) 요구 사항
    - 메타 신규 거래로그 조회(표준 스키마/메타데이터 정합성)
    - 권한/승인 관리 프로세스(요청-검토-승인-만료) 개발
    - 권한별 화면제어, 공통코드 유효값 연동, 로그 상세 화면 설계
다. 구축 기간: 2024년 10월 ~ 2024년 12월(예정)
라. 비고: 상세 범위·예산은 본 RFP 본문에서 확정 예정(현행 연계/SSO/IAM·보안 정책 준수)
"""

# 제안서 공통 구조(사용자가 미리 정한 일반 구조)를 간략화하여 유지
instructions = """
<출력 형식>
- 첫 줄: 제안서: 신한투자증권 로그정보시스템 고도화 – 안정성 중시형 제안(가상)
- 둘째 줄: 제안사: [가상 회사명], 작성일/담당자: [간략 표기]

<제안서 구조(요약형)>
1. 제안 개요(배경/목표/KPI 개괄)
2. 제안사 개요 및 유사 수행 경험(간단 요약)
3. 프로젝트 수행
   3.1 기술 부문: 시스템 구축 전략, 구성, 성능/테스트/보안(원론적 기술)
   3.2 일반 부문: 일정, 인력(역할만), 방법론/도구(대표 키워드만)
4. 사업 관리: 관리/품질/보고(원칙 위주), 교육/이행/유지보수(간단 계획)
5. 결론: 안정성·리스크 관리 강조, 세부는 협의 후 보강 예정

<제안서 세부 작성 방법(보통퀄리티·안정성 중시형·8항목)>
1) 문체는 보수적·형식적으로 유지하고, ‘검증된 기술·절차’ 중심의 표현 사용.
2) RFP 요구사항은 핵심만 선별 제시(일부 항목은 “추후 협의·세부화 예정”).
3) 기술 스택은 범용·안정 조합 위주로, 선택 사유는 간단 근거만 제시.
4) 성능/가용성/보안은 최소 기준 선언(P95 응답, 가용성, 접근통제 ‘적용 예정’ 수준).
5) 테스트/품질/검수는 표준 산출물·체크리스트 언급만(세부 케이스/수치 생략).
6) 일정/조직은 고수준 타임라인·핵심 역할만(세부 R&R/캘린더 생략).
7) 리스크 관리는 대표 항목만 표로 제시(완화 전략은 개괄적 표현).
8) 규정·컴플라이언스·감사 추적은 ‘현행 준수 원칙’과 “정책 반영 예정” 수준으로 표기.
"""

# 프롬프트 결합
prompt = f"{persona}\n{concept}\n{goal_context}\n{quality_constraints}\n{instructions}"

# 최신 openai 라이브러리 방식 (openai>=1.0.0)
client = OpenAI(api_key=api_key)
response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {"role": "user", "content": prompt}
    ]
)



API Key loaded


In [5]:
# 답변 출력
try:
    print(response.choices[0].message.content)
except Exception as e:
    try:
        print(response.output_text)
    except Exception:
        print(f"출력 에러: {e}")

제안서: 신한투자증권 로그정보시스템 고도화 – 안정성 중시형 제안(가상)
제안사: 안정플랜테크(가상), 작성일/담당자: 2025-10-01 / 이민석 PM

1. 제안 개요(배경/목표/KPI 개괄)
- 배경
  - 메타 차세대 프로젝트와의 연계를 통해 신규 로그를 이해하기 쉬운 웹 UI로 조회할 수 있도록 시스템을 고도화하고, 로그 관리의 컴플라이언스 강화 및 그룹 권한에 따른 차등 정보 노출을 구현하고자 함.
- 목표
  - 사용자 중심 UX를 통해 로그 탐색·검색·필터링의 경험을 개선
  - 민감정보 항목 승인/권한 관리 및 감사 추적을 강화하여 컴플라이언스 이슈에 대응
  - 그룹 권한(IT/일반직원 등)에 따른 로그 조회 제한과 차등 정보 노출 구조를 확립
- KPI 개괄
  - 시스템 안정성과 가용성의 현행 수준 유지 및 향상 가능성 평가
  - P95 응답시간 현행 대비 유사 수준/향상 여지 확보(구체 수치는 추후 협의)
  - 감사 추적 및 권한 관리 정책의 적용 여부(정책 반영 예정) 및 운용성 확보
  - 사용자 만족도 및 운영 편의성 개선의 정성적 지표(추후 구체화 예정)

2. 제안사 개요 및 유사 수행 경험(간단 요약)
- 제안사 개요
  - 다년간 금융권 대규모 시스템의 안정성 중심 설계 및 운영 경험을 축적한 보수적이고 재현가능한 품질 체계를 갖춘 엔지니어링 파트너.
- 유사 수행 경험(간단 요약)
  - 금융권 로그 관리/감사 시스템의 안정성 강화 프로젝트 다수 수행
  - RBAC/권한 관리, 감사 로그 수집 및 보관 체계 도입 사례
  - 웹 UI 기반 로그 조회 대시보드의 안정적 운영 및 규정 준수 체계 구축 경험

3. 프로젝트 수행
3.1 기술 부문
- 시스템 구축 전략
  - 다층 아키텍처 기반의 안정성 중심 설계
  - 현행 정책 준수 원칙을 반영하는 감사 로그 중심의 설계
- 구성(고수준)
  - 프런트엔드: 안정적 웹 UI via 현대 프레임워크
  - 백엔드: 표준화된 API 계층(Spring Boot 기반)
  -

In [10]:
import re

def markdown_to_structured_html(markdown_text):
    """
    Markdown 텍스트를 구조화된 HTML로 변환
    Upstage OCR Parser의 출력 형식과 유사한 계층 구조 생성
    """
    html_lines = ['<!DOCTYPE html>', '<html>', '<head>', 
                  '<meta charset="UTF-8">', 
                  '<title>제안서</title>',
                  '<style>',
                  'body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; }',
                  'h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }',
                  'h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }',
                  'h3 { color: #7f8c8d; margin-top: 20px; }',
                  'table { border-collapse: collapse; width: 100%; margin: 20px 0; }',
                  'th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }',
                  'th { background-color: #3498db; color: white; }',
                  'ul, ol { margin: 10px 0; padding-left: 30px; }',
                  'p { margin: 10px 0; }',
                  '</style>',
                  '</head>', '<body>']
    
    lines = markdown_text.split('\n')
    in_table = False
    in_list = False
    
    for line in lines:
        line = line.strip()
        
        if not line:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<br/>')
            continue
        
        # 제목 변환
        if line.startswith('# '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h1>{line[2:]}</h1>')
        elif line.startswith('## '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h2>{line[3:]}</h2>')
        elif line.startswith('### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h3>{line[4:]}</h3>')
        elif line.startswith('#### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h4>{line[5:]}</h4>')
        
        # 표 감지 (| ... | 형태)
        elif '|' in line and not in_table:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<table>')
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<th>{cell}</th>')
            html_lines.append('</tr>')
            in_table = True
        elif '|' in line and in_table:
            if '---' in line:  # 구분선 무시
                continue
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<td>{cell}</td>')
            html_lines.append('</tr>')
        elif in_table and '|' not in line:
            html_lines.append('</table>')
            in_table = False
        
        # 목록 변환
        elif line.startswith('- ') or line.startswith('* '):
            if not in_list:
                html_lines.append('<ul>')
                in_list = True
            content = line[2:].strip()
            # **굵게** 변환
            content = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', content)
            html_lines.append(f'<li>{content}</li>')
        
        # 일반 텍스트
        else:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            # **굵게** 변환
            line = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', line)
            html_lines.append(f'<p>{line}</p>')
    
    if in_table:
        html_lines.append('</table>')
    if in_list:
        html_lines.append('</ul>')
    
    html_lines.extend(['</body>', '</html>'])
    
    return '\n'.join(html_lines)

# 제안서를 구조화된 HTML로 변환
try:
    proposal_text = response.choices[0].message.content
except:
    proposal_text = response.output_text

structured_html = markdown_to_structured_html(proposal_text)

# HTML 파일로 저장
output_path = 'proposal_good_4of5.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(structured_html)

print(f"구조화된 HTML 제안서가 생성되었습니다: {output_path}")
print(f"총 {len(structured_html)} 문자")
print(structured_html)

구조화된 HTML 제안서가 생성되었습니다: proposal_good_4of5.html
총 3053 문자
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>제안서</title>
<style>
body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; }
h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }
h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }
h3 { color: #7f8c8d; margin-top: 20px; }
table { border-collapse: collapse; width: 100%; margin: 20px 0; }
th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }
th { background-color: #3498db; color: white; }
ul, ol { margin: 10px 0; padding-left: 30px; }
p { margin: 10px 0; }
</style>
</head>
<body>
<p>제안서: 신한투자증권 로그정보시스템 고도화 – 안정성 중시형 제안(가상)</p>
<p>제안사: AegisGuard Technologies (가상), 작성일/담당자: 2024-10-01 / 박상윤 책임</p>
<br/>
<p>1. 제안 개요(배경/목표/KPI 개괄)</p>
<ul>
<li>배경: 메타 차세대 프로젝트 연계 로그의 사용자 이해도 향상 및 컴플라이언스 요구 강화 필요.</li>
<li>목표: 안정적 시스템 고도화로 사용자 중심의 로그 조회·필터링 제공, 권한 승인 관리 및 감사 추적 강화, 그룹